### v12 : fix bug
### v11 : drop more features based on feature importances
### v10 : drop some features based on feature importances
### v9 : add feature importance plot
### v8 : add comments
### v7 : 0.01387

I simply replace my neural network model with lgbm model from https://www.kaggle.com/mrkmakr/neural-network-with-mae-objective-0-01385

Some feature engineerings (standerize, categorical version of numeric feature) used in this kernel are not suitable to lgbm.

I just use almost the same features used in the above kernel

In this kernel, I use the multiple LGBMClassifiers.

Target Yard of lgbm is smoothened and fed to LGBMClassifier. 

- For example, (1,2,3,4,5) -> label 1, (6,7,8,9,10) -> label 2, etc...

And I slide the smoothing window and train each LGBMClassifier.

- For example, 
- First model : (1,2,3,4,5) -> label 1, (6,7,8,9,10) -> label 2, etc...
- Second model : (2,3,4,5,6) -> label 1, (7,8,9,10,11) -> label 2, etc...
- Third model : (3,4,5,6,7) -> label 1, (8,9,10,11,12) -> label 2, etc...
- etc...

Finaly I calculate the ensemble of predictions of these multiple LGBMClassifiers.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime

from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [15,10]

In [42]:
from sklearn.ensemble import RandomForestRegressor,VotingClassifier, VotingRegressor
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold

In [3]:
# Model NN Keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import Callback, EarlyStopping
from keras import backend as K
from keras import regularizers
import tensorflow as tf

Using TensorFlow backend.


In [4]:
#train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv', dtype={'WindSpeed': 'object'})
train = pd.read_csv('../data/train.csv', dtype={'WindSpeed': 'object'})
#train = train[:2200]
print(train.shape)
train.head()

(509762, 49)


,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


## Feature engineering

In [5]:
#https://www.kaggle.com/rooshroosh/fork-of-neural-networks-different-architecture
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

def strtofloat(x):
    try:
        return float(x)
    except:
        return -1

def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def orientation_to_cat(x):
    x = np.clip(x, 0, 360 - 1)
    try:
        return str(int(x/15))
    except:
        return "nan"

In [6]:
def preprocess(train):
    ## GameClock
    train['GameClock_sec'] = train['GameClock'].apply(strtoseconds)
    train["GameClock_minute"] = train["GameClock"].apply(lambda x : x.split(":")[0]).astype("object")

    ## Height
    train['PlayerHeight_dense'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

    ## Time
    train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

    train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

    ## Age
    seconds_in_year = 60*60*24*365.25
    train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    train["PlayerAge_ob"] = train['PlayerAge'].astype(np.int).astype("object")

    ## WindSpeed
    train['WindSpeed_ob'] = train['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    train['WindSpeed_ob'] = train['WindSpeed_ob'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    train['WindSpeed_ob'] = train['WindSpeed_ob'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    train['WindSpeed_dense'] = train['WindSpeed_ob'].apply(strtofloat)

    ## Weather
    train['GameWeather_process'] = train['GameWeather'].str.lower()
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: "indoor" if not pd.isna(x) and "indoor" in x else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    train['GameWeather_process'] = train['GameWeather_process'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    train['GameWeather_dense'] = train['GameWeather_process'].apply(map_weather)

    ## Rusher
    train['IsRusher'] = (train['NflId'] == train['NflIdRusher'])
    train['IsRusher_ob'] = (train['NflId'] == train['NflIdRusher']).astype("object")
    temp = train[train["IsRusher"]][["Team", "PlayId"]].rename(columns={"Team":"RusherTeam"})
    train = train.merge(temp, on = "PlayId")
    train["IsRusherTeam"] = train["Team"] == train["RusherTeam"]

    ## dense -> categorical
    train["Quarter_ob"] = train["Quarter"].astype("object")
    train["Down_ob"] = train["Down"].astype("object")
    train["JerseyNumber_ob"] = train["JerseyNumber"].astype("object")
    train["YardLine_ob"] = train["YardLine"].astype("object")
    # train["DefendersInTheBox_ob"] = train["DefendersInTheBox"].astype("object")
    # train["Week_ob"] = train["Week"].astype("object")
    # train["TimeDelta_ob"] = train["TimeDelta"].astype("object")


    ## Orientation and Dir
    train["Orientation_ob"] = train["Orientation"].apply(lambda x : orientation_to_cat(x)).astype("object")
    train["Dir_ob"] = train["Dir"].apply(lambda x : orientation_to_cat(x)).astype("object")

    train["Orientation_sin"] = train["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    train["Orientation_cos"] = train["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    train["Dir_sin"] = train["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    train["Dir_cos"] = train["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

    ## diff Score
    train["diffScoreBeforePlay"] = train["HomeScoreBeforePlay"] - train["VisitorScoreBeforePlay"]
    train["diffScoreBeforePlay_binary_ob"] = (train["HomeScoreBeforePlay"] > train["VisitorScoreBeforePlay"]).astype("object")

    ## Turf
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    train['Turf'] = train['Turf'].map(Turf)

    ## OffensePersonnel
    temp = train["OffensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(OffensePersonnelSplit(x)))
    temp.columns = ["Offense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = "PlayId")

    ## DefensePersonnel
    temp = train["DefensePersonnel"].iloc[np.arange(0, len(train), 22)].apply(lambda x : pd.Series(DefensePersonnelSplit(x)))
    temp.columns = ["Defense" + c for c in temp.columns]
    temp["PlayId"] = train["PlayId"].iloc[np.arange(0, len(train), 22)]
    train = train.merge(temp, on = "PlayId")

    ## sort
#     train = train.sort_values(by = ['X']).sort_values(by = ['Dis']).sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index(drop = True)
    train = train.sort_values(by = ['X']).sort_values(by = ['Dis']).sort_values(by=['PlayId', 'IsRusherTeam', 'IsRusher']).reset_index(drop = True)
    return train

In [7]:
%%time
train = preprocess(train)

CPU times: user 2min 8s, sys: 5.13 s, total: 2min 13s
Wall time: 1min 58s


In [8]:
## DisplayName remove Outlier
v = train["DisplayName"].value_counts()
missing_values = list(v[v < 5].index)
train["DisplayName"] = train["DisplayName"].where(~train["DisplayName"].isin(missing_values), "nan")

## PlayerCollegeName remove Outlier
v = train["PlayerCollegeName"].value_counts()
missing_values = list(v[v < 10].index)
train["PlayerCollegeName"] = train["PlayerCollegeName"].where(~train["PlayerCollegeName"].isin(missing_values), "nan")

In [9]:
pd.to_pickle(train, "../data/train.pkl")

In [10]:
def drop(train):
    drop_cols = ["GameId", "GameWeather", "NflId", "Season", "NflIdRusher"] 
    drop_cols += ['TimeHandoff', 'TimeSnap', 'PlayerBirthDate']
    drop_cols += ["Orientation", "Dir", 'WindSpeed', "GameClock"]
    # drop_cols += ["DefensePersonnel","OffensePersonnel"]
    train = train.drop(drop_cols, axis = 1)
    return train

In [11]:
train = drop(train)

In [12]:
un_use_features = ['G_HomeTeamAbbr',
 'G_PossessionTeam',
 'G_RusherTeam',
 'G_StadiumType',
 'G_diffScoreBeforePlay_binary_ob',
 'P_DefenseDB',
 'P_DefenseDL',
 'P_DefenseLB',
 'P_DefenseOL',
 'P_GameWeather_dense',
 'P_IsRusher',
 'P_IsRusherTeam',
 'P_OffenseDB',
 'P_OffenseDL',
 'P_OffenseLB',
 'P_OffenseOL',
 'P_OffenseQB',
 'P_OffenseTE',
 'P_OffenseWR',
 'P_PlayerAge_ob',
 'P_Quarter',
 'P_Week']

un_use_features += ['G_Down_ob',
 'G_FieldPosition',
 'G_OffenseRB',
 'G_TimeDelta',
 'G_VisitorTeamAbbr',
 'G_WindDirection',
 'P_GameClock_sec',
 'P_HomeScoreBeforePlay',
 'P_Humidity',
 'P_Orientation_ob',
 'P_PlayerHeight',
 'P_Temperature',
 'P_VisitorScoreBeforePlay',
 'P_WindSpeed_dense',
 'P_diffScoreBeforePlay']

## delete prefix
un_use_features = [c[2:] for c in un_use_features]
train = train.drop(un_use_features, axis = 1)

In [13]:
cat_features = []
dense_features = []
for col in train.columns:
    if train[col].dtype =='object':
        cat_features.append(col)
        print("*cat*", col, len(train[col].unique()))
    else:
        dense_features.append(col)
        print("!dense!", col, len(train[col].unique()))
dense_features.remove("PlayId")
dense_features.remove("Yards")

!dense! PlayId 23171
*cat* Team 2
!dense! X 10890
!dense! Y 4339
!dense! S 884
!dense! A 903
!dense! Dis 105
*cat* DisplayName 2128
!dense! JerseyNumber 99
!dense! YardLine 50
!dense! Down 4
!dense! Distance 35
*cat* OffenseFormation 9
*cat* OffensePersonnel 56
!dense! DefendersInTheBox 12
*cat* DefensePersonnel 38
*cat* PlayDirection 2
!dense! Yards 94
!dense! PlayerWeight 182
*cat* PlayerCollegeName 291
*cat* Position 25
*cat* Stadium 55
*cat* Location 60
*cat* Turf 2
*cat* GameClock_minute 16
!dense! PlayerHeight_dense 16
!dense! PlayerAge 506270
*cat* WindSpeed_ob 34
*cat* GameWeather_process 42
*cat* IsRusher_ob 2
*cat* Quarter_ob 5
*cat* JerseyNumber_ob 99
*cat* YardLine_ob 50
*cat* Dir_ob 25
!dense! Orientation_sin 32104
!dense! Orientation_cos 32567
!dense! Dir_sin 32104
!dense! Dir_cos 32567


## categorical

In [14]:
train_cat = train[cat_features]
categories = []
most_appear_each_categories = {}
for col in tqdm_notebook(train_cat.columns):
    train_cat.loc[:,col] = train_cat[col].fillna("nan")
    train_cat.loc[:,col] = col + "__" + train_cat[col].astype(str)
    most_appear_each_categories[col] = list(train_cat[col].value_counts().index)[0]
    categories.append(train_cat[col].unique())
categories = np.hstack(categories)
print(len(categories))


2941


In [15]:
le = LabelEncoder()
le.fit(categories)
for col in tqdm_notebook(train_cat.columns):
    train_cat.loc[:, col] = le.transform(train_cat[col])
num_classes = len(le.classes_)

## Dense

In [16]:
train_dense = train[dense_features]
sss = {}
medians = {}
for col in tqdm_notebook(train_dense.columns):
    print(col)
    medians[col] = np.nanmedian(train_dense[col])
    train_dense.loc[:, col] = train_dense[col].fillna(medians[col])
    ss = StandardScaler()
    train_dense.loc[:, col] = ss.fit_transform(train_dense[col].values[:,None])
    sss[col] = ss

X
Y
S
A
Dis
JerseyNumber
YardLine
Down
Distance
DefendersInTheBox
PlayerWeight
PlayerHeight_dense
PlayerAge
Orientation_sin
Orientation_cos
Dir_sin
Dir_cos



## Divide features into groups

In [17]:
eps = 1e-8
## dense features for play
dense_game_features = train_dense.columns[train_dense[:22].std() <= eps]
## dense features for each player
dense_player_features = train_dense.columns[train_dense[:22].std() > eps]
## categorical features for play
cat_game_features = train_cat.columns[train_cat[:22].std() <= eps]
## categorical features for each player
cat_player_features = train_cat.columns[train_cat[:22].std() > eps]

In [18]:
dense_game_feature_names = ["G_" + cc for cc in dense_game_features]
dense_player_feature_names = list(np.hstack([["P_" + c for c in dense_player_features] for k in range(22)]))
cat_game_feature_names = ["G_" + cc for cc in cat_game_features]
cat_player_feature_names = list(np.hstack([["P_" + c for c in cat_player_features] for k in range(22)]))

In [19]:
train_dense_game = train_dense[dense_game_features].iloc[np.arange(0, len(train), 22)].reset_index(drop = True).values
## rusher player feature is included in train_dense_players, so skip this.
# train_dense_game = np.hstack([train_dense_game, train_dense[dense_player_features][train_dense["IsRusher"] > 0]]) ## with rusher player feature

train_dense_players = [train_dense[dense_player_features].iloc[np.arange(k, len(train), 22)].reset_index(drop = True) for k in range(22)]
train_dense_players = np.stack([t.values for t in train_dense_players]).transpose(1, 0, 2)

train_cat_game = train_cat[cat_game_features].iloc[np.arange(0, len(train), 22)].reset_index(drop = True).values
# train_cat_game = np.hstack([train_cat_game, train_cat[cat_player_features][train_dense["IsRusher"] > 0]]) ## with rusher player feature

train_cat_players = [train_cat[cat_player_features].iloc[np.arange(k, len(train), 22)].reset_index(drop = True) for k in range(22)]
train_cat_players = np.stack([t.values for t in train_cat_players]).transpose(1, 0, 2)

In [20]:
def return_step(x):
    temp = np.zeros(199)
    temp[x + 99:] = 1
    return temp

train_y_raw = train["Yards"].iloc[np.arange(0, len(train), 22)].reset_index(drop = True)
train_y = np.vstack(train_y_raw.apply(return_step).values)

In [21]:
train_dense_game.shape, train_dense_players.shape, train_cat_game.shape, train_cat_players.shape, train_y.shape

((23171, 4), (23171, 22, 13), (23171, 12), (23171, 22, 7), (23171, 199))

# Let's build models

In [22]:
models = []
losses = []

### LGBM Model

In [23]:
## concat all features
train_dense_players_lgb = np.reshape(train_dense_players, (len(train_dense_players), -1))
train_dense = np.hstack([train_dense_players_lgb, train_dense_game])

train_cat_players_lgb = np.reshape(train_cat_players, (len(train_cat_players), -1))
train_cat = np.hstack([train_cat_players_lgb, train_cat_game])

train_x = np.hstack([train_dense, train_cat])

In [24]:
train_x.shape

(23171, 456)

In [25]:
class MultiLGBMClassifier():
    def __init__(self, resolution, params):
        ## smoothing size
        self.resolution = resolution
        ## initiarize models
        self.models = [LGBMClassifier(**params) for _ in range(resolution)]
        
    def fit(self, x, y):
        self.classes_list = []
        for k in tqdm_notebook(range(self.resolution)):
            ## train each model
            self.models[k].fit(x, (y + k) // self.resolution)
            ## (0,1,2,3,4,5,6,7,8,9) -> (0,0,0,0,0,1,1,1,1,1) -> (0,5)
            classes = np.sort(list(set((y + k) // self.resolution))) * self.resolution - k
            classes = np.append(classes, 999)
            self.classes_list.append(classes)
            
    def predict(self, x):
        pred199_list = []
        for k in range(self.resolution):
            preds = self.models[k].predict_proba(x)
            classes = self.classes_list[k]
            pred199s = self.get_pred199(preds, classes)
            pred199_list.append(pred199s)
        self.pred199_list = pred199_list
        pred199_ens = np.mean(np.stack(pred199_list), axis = 0)
        return pred199_ens
    
    def _get_pred199(self, p, classes):
        ## categorical prediction -> predicted distribution whose length is 199
        pred199 = np.zeros(199)
        for k in range(len(p)):
            pred199[classes[k] + 99 : classes[k+1] + 99] = p[k]
        return pred199

    def get_pred199(self, preds, classes):
        pred199s = []
        for p in preds:
            pred199 = np.cumsum(self._get_pred199(p, classes))
            pred199 = pred199/np.max(pred199)
            pred199s.append(pred199)
        return np.vstack(pred199s)

In [26]:
params = {'lambda_l1': 0.001,'lambda_l2': 0.001,'num_leaves': 40,'feature_fraction': 0.4,
          'subsample': 0.4, 'min_child_samples': 10,'learning_rate': 0.01,'num_iterations': 700, 
          'random_state': 42}

In [27]:
for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_y)):
        print("-----------")
        print("-----------")
        model_LGB = MultiLGBMClassifier(resolution = 5, params = params)
        model_LGB.fit(train_x[tr_inds], train_y_raw.values[tr_inds])
        preds = model_LGB.predict(train_x[val_inds])
        loss = np.mean((train_y[val_inds] - preds) ** 2)
        models.append(model_LGB)
        print(k_fold, loss)
        losses.append(loss)
print("-------")
print(losses)
print(np.mean(losses))

-----------
-----------



0 0.01318628130402307
-----------
-----------



1 0.013097719247489224
-----------
-----------



2 0.012975416704064999
-----------
-----------



3 0.012995066903846556
-----------
-----------



4 0.013439471590426997
-------
[0.01318628130402307, 0.013097719247489224, 0.012975416704064999, 0.012995066903846556, 0.013439471590426997]
0.013138791149970169


In [28]:
print(losses)
print(np.mean(losses))

[0.01318628130402307, 0.013097719247489224, 0.012975416704064999, 0.012995066903846556, 0.013439471590426997]
0.013138791149970169


### Neural Network Model

In [29]:
keras.backend.clear_session()
def crps(y_true, y_pred):
    loss = K.mean((K.cumsum(y_pred, axis = 1) - y_true)**2)
    return loss

def get_model(batch_size, epochs):
    ## model dense
    input_dense_game = keras.layers.Input(shape=(train_dense_game.shape[1],))
    x1 = keras.layers.Dense(32, activation="relu")(input_dense_game)
    x1 = keras.layers.Dropout(0.5)(x1)
    # x1 = keras.layers.Dropout(0.1)(x1)

    input_dense_players = keras.layers.Input(shape=(train_dense_players.shape[1],train_dense_players.shape[2]))
    x2 = keras.layers.Dense(32, activation="relu")(input_dense_players)
    x2 = keras.layers.Dropout(0.5)(x2)
    # x2 = keras.layers.Flatten()(x2)
    # x2 = keras.layers.Dropout(0.1)(x2)

    ## model categorical
    input_cat_game = keras.layers.Input(shape=(train_cat_game.shape[1], ))
    embedding = keras.layers.Embedding(num_classes, 8, embeddings_regularizer=regularizers.l2(1))

    x3 = embedding(input_cat_game)
    x3 = keras.layers.Flatten()(x3)
    x3 = keras.layers.Dense(8, activation="relu")(x3)
    x3 = keras.layers.Dropout(0.6)(x3)

    input_cat_players = keras.layers.Input(shape=(train_cat_players.shape[1], train_cat_players.shape[2]))
    x4 = embedding(input_cat_players)

    x4 = keras.layers.Reshape((int(x4.shape[1]), int(x4.shape[2]) * int(x4.shape[3])))(x4)
    x4 = keras.layers.Dense(16, activation="relu")(x4)
    x4 = keras.layers.Dropout(0.6)(x4)

    ### concat players
    x_concat_players = keras.layers.Concatenate()([x2,x4])
    x_concat_players = keras.layers.Dense(16, activation="relu")(x_concat_players)
    # x_concat_players = keras.layers.GlobalAveragePooling1D()(x_concat_players)

    ## flatten
    x2 = keras.layers.Flatten()(x2)
    x4 = keras.layers.Flatten()(x4)
    x_concat_players = keras.layers.Flatten()(x_concat_players)

    ### concat all
    x_concat = keras.layers.Concatenate()([x1,x3,x_concat_players] + [x2, x4])
    x_concats = []
    n_unit = 128
    decay_rate = 0.5
    for k in range(5):
        x_concat = keras.layers.Dense(n_unit, activation="relu")(x_concat)
        x_concats.append(x_concat)
        n_unit = int(n_unit * decay_rate)
    x_concat = keras.layers.Concatenate()(x_concats)
    x_concat = keras.layers.Dropout(0.5)(x_concat)

    ## concat
    x_concat = keras.layers.Concatenate()([x1,x3,x_concat_players,x_concat] + [x2, x4])
    out_soft = keras.layers.Dense(199, activation="softmax", name = "out_soft")(x_concat)
    out_reg = keras.layers.Dense(1, activation=None, name = "out_reg")(x_concat)
    modelNN = keras.models.Model(inputs = [input_dense_game, input_dense_players, input_cat_game, input_cat_players],
                               outputs = [out_soft, out_reg])

    ## compile
    er = EarlyStopping(patience=10, min_delta=1e-4, restore_best_weights=True, monitor='val_out_soft_loss')
    modelNN.compile(loss=[crps, keras.losses.mae],
                  loss_weights=[1.0, 0.01],
                  optimizer=keras.optimizers.Adam(learning_rate=0.001, decay = 1e-5))

    ## train
    tr_x = [train_dense_game[tr_inds], train_dense_players[tr_inds], train_cat_game[tr_inds], train_cat_players[tr_inds]]
    tr_y = [train_y[tr_inds], train_y_raw[tr_inds]/100]
    val_x = [train_dense_game[val_inds], train_dense_players[val_inds], train_cat_game[val_inds], train_cat_players[val_inds]]
    val_y = [train_y[val_inds], train_y_raw[val_inds]/100]
    modelNN.fit(tr_x,
              tr_y,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(val_x, val_y),
              callbacks=[er]
             )
    loss = modelNN.history.history["val_out_soft_loss"][-1]
    return modelNN, loss


In [34]:
for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_y)):
        #print("-----------")
        modelNN, loss = get_model(1024, 250)    
        #if (loss < 0.013):
            #print('Append KFold:',k_fold,'| Loss:', loss)
        models.append(modelNN)
        losses.append(loss)
            #if (len(losses) > 10):
                #break
        #else:
            #print('Ignore KFold:',k_fold, '| Loss:', loss)        
print("-------")
print('Loss Mean:', np.mean(losses), ' | List:', losses)

Train on 18536 samples, validate on 4635 samples
Epoch 1/250
18536/18536 [==============================] - 2s 133us/step - loss: 11.7873 - out_soft_loss: 0.0263 - out_reg_loss: 0.6204 - val_loss: 5.0598 - val_out_soft_loss: 0.0146 - val_out_reg_loss: 0.2574
Epoch 2/250
18536/18536 [==============================] - 2s 82us/step - loss: 2.6231 - out_soft_loss: 0.0144 - out_reg_loss: 0.4237 - val_loss: 0.8226 - val_out_soft_loss: 0.0142 - val_out_reg_loss: 0.1705
Epoch 3/250
18536/18536 [==============================] - 1s 80us/step - loss: 0.3669 - out_soft_loss: 0.0141 - out_reg_loss: 0.2831 - val_loss: 0.0854 - val_out_soft_loss: 0.0140 - val_out_reg_loss: 0.1251
Epoch 4/250
18536/18536 [==============================] - 1s 81us/step - loss: 0.0424 - out_soft_loss: 0.0140 - out_reg_loss: 0.1931 - val_loss: 0.0202 - val_out_soft_loss: 0.0140 - val_out_reg_loss: 0.0852
Epoch 5/250
18536/18536 [==============================] - 1s 81us/step - loss: 0.0184 - out_soft_loss: 0.0140 - out_

Epoch 3/250
18537/18537 [==============================] - 2s 85us/step - loss: 0.3638 - out_soft_loss: 0.0143 - out_reg_loss: 0.2711 - val_loss: 0.0840 - val_out_soft_loss: 0.0139 - val_out_reg_loss: 0.1079
Epoch 4/250
18537/18537 [==============================] - 2s 87us/step - loss: 0.0422 - out_soft_loss: 0.0140 - out_reg_loss: 0.1867 - val_loss: 0.0202 - val_out_soft_loss: 0.0138 - val_out_reg_loss: 0.0772
Epoch 5/250
18537/18537 [==============================] - 2s 92us/step - loss: 0.0186 - out_soft_loss: 0.0140 - out_reg_loss: 0.1366 - val_loss: 0.0162 - val_out_soft_loss: 0.0138 - val_out_reg_loss: 0.0886
Epoch 6/250
18537/18537 [==============================] - 2s 95us/step - loss: 0.0158 - out_soft_loss: 0.0140 - out_reg_loss: 0.1088 - val_loss: 0.0150 - val_out_soft_loss: 0.0137 - val_out_reg_loss: 0.0949
Epoch 7/250
18537/18537 [==============================] - 2s 88us/step - loss: 0.0149 - out_soft_loss: 0.0138 - out_reg_loss: 0.0944 - val_loss: 0.0145 - val_out_soft_

Epoch 4/250
18537/18537 [==============================] - 2s 81us/step - loss: 0.0424 - out_soft_loss: 0.0143 - out_reg_loss: 0.1502 - val_loss: 0.0202 - val_out_soft_loss: 0.0138 - val_out_reg_loss: 0.0885
Epoch 5/250
18537/18537 [==============================] - 2s 85us/step - loss: 0.0183 - out_soft_loss: 0.0139 - out_reg_loss: 0.1133 - val_loss: 0.0159 - val_out_soft_loss: 0.0137 - val_out_reg_loss: 0.0725
Epoch 6/250
18537/18537 [==============================] - 2s 83us/step - loss: 0.0156 - out_soft_loss: 0.0141 - out_reg_loss: 0.0951 - val_loss: 0.0145 - val_out_soft_loss: 0.0137 - val_out_reg_loss: 0.0615
Epoch 7/250
18537/18537 [==============================] - 2s 81us/step - loss: 0.0148 - out_soft_loss: 0.0140 - out_reg_loss: 0.0888 - val_loss: 0.0144 - val_out_soft_loss: 0.0136 - val_out_reg_loss: 0.0738
Epoch 8/250
18537/18537 [==============================] - 2s 85us/step - loss: 0.0146 - out_soft_loss: 0.0137 - out_reg_loss: 0.0737 - val_loss: 0.0141 - val_out_soft_

18537/18537 [==============================] - 2s 87us/step - loss: 0.0142 - out_soft_loss: 0.0137 - out_reg_loss: 0.0508 - val_loss: 0.0138 - val_out_soft_loss: 0.0134 - val_out_reg_loss: 0.0386
Epoch 15/250
18537/18537 [==============================] - 2s 86us/step - loss: 0.0141 - out_soft_loss: 0.0135 - out_reg_loss: 0.0493 - val_loss: 0.0138 - val_out_soft_loss: 0.0134 - val_out_reg_loss: 0.0382
Epoch 16/250
18537/18537 [==============================] - 2s 87us/step - loss: 0.0141 - out_soft_loss: 0.0137 - out_reg_loss: 0.0482 - val_loss: 0.0138 - val_out_soft_loss: 0.0134 - val_out_reg_loss: 0.0428
Epoch 17/250
18537/18537 [==============================] - 2s 102us/step - loss: 0.0141 - out_soft_loss: 0.0135 - out_reg_loss: 0.0484 - val_loss: 0.0137 - val_out_soft_loss: 0.0134 - val_out_reg_loss: 0.0372
Epoch 18/250
18537/18537 [==============================] - 2s 97us/step - loss: 0.0140 - out_soft_loss: 0.0136 - out_reg_loss: 0.0462 - val_loss: 0.0137 - val_out_soft_loss: 0

18537/18537 [==============================] - 2s 84us/step - loss: 0.0137 - out_soft_loss: 0.0132 - out_reg_loss: 0.0419 - val_loss: 0.0141 - val_out_soft_loss: 0.0139 - val_out_reg_loss: 0.0434
Epoch 27/250
18537/18537 [==============================] - 2s 86us/step - loss: 0.0137 - out_soft_loss: 0.0132 - out_reg_loss: 0.0408 - val_loss: 0.0140 - val_out_soft_loss: 0.0138 - val_out_reg_loss: 0.0375
Epoch 28/250
18537/18537 [==============================] - 2s 84us/step - loss: 0.0137 - out_soft_loss: 0.0132 - out_reg_loss: 0.0410 - val_loss: 0.0140 - val_out_soft_loss: 0.0138 - val_out_reg_loss: 0.0378
Epoch 29/250
18537/18537 [==============================] - 2s 82us/step - loss: 0.0137 - out_soft_loss: 0.0132 - out_reg_loss: 0.0400 - val_loss: 0.0141 - val_out_soft_loss: 0.0138 - val_out_reg_loss: 0.0419
Epoch 30/250
18537/18537 [==============================] - 2s 83us/step - loss: 0.0137 - out_soft_loss: 0.0132 - out_reg_loss: 0.0414 - val_loss: 0.0140 - val_out_soft_loss: 0.

In [35]:
print(losses)
print(np.mean(losses))

[0.013419183902442455, 0.013236845843493938, 0.013268450275063515, 0.01330277044326067, 0.0138024240732193]
0.013405934907495975


In [36]:
models

In [45]:
#modelVT = VotingClassifier(models, voting='soft')
#modelVT.fit(train_x, train_y)
#res_y = modelVT.predict(train_x)

er = VotingRegressor(models)
#print(er.fit(train_x, train_y).predict(train_x))

In [50]:
er.fit(train_x, train_y)

TypeError: zip argument #1 must support iteration

## Prediction

In [ ]:
def make_pred(test, sample, env, model):
    test = preprocess(test)
    test = drop(test)
    test = test.drop(un_use_features, axis = 1)
    
    ### categorical
    test_cat = test[cat_features]
    for col in (test_cat.columns):
        test_cat.loc[:,col] = test_cat[col].fillna("nan")
        test_cat.loc[:,col] = col + "__" + test_cat[col].astype(str)
        isnan = ~test_cat.loc[:,col].isin(categories)
        if np.sum(isnan) > 0:
#             print("------")
#             print("test have unseen label : col")
            if not ((col + "__nan") in categories):
#                 print("not nan in train : ", col)
                test_cat.loc[isnan,col] = most_appear_each_categories[col]
            else:
#                 print("nan seen in train : ", col)
                test_cat.loc[isnan,col] = col + "__nan"
    for col in (test_cat.columns):
        test_cat.loc[:, col] = le.transform(test_cat[col])

    ### dense
    test_dense = test[dense_features]
    for col in (test_dense.columns):
        test_dense.loc[:, col] = test_dense[col].fillna(medians[col])
        test_dense.loc[:, col] = sss[col].transform(test_dense[col].values[:,None])

    ### divide
    test_dense_players = [test_dense[dense_player_features].iloc[np.arange(k, len(test), 22)].reset_index(drop = True) for k in range(22)]
    test_dense_players = np.stack([t.values for t in test_dense_players]).transpose(1,0, 2)

    test_dense_game = test_dense[dense_game_features].iloc[np.arange(0, len(test), 22)].reset_index(drop = True).values
#     test_dense_game = np.hstack([test_dense_game, test_dense[dense_player_features][test_dense["IsRusher"] > 0]])
    
    test_cat_players = [test_cat[cat_player_features].iloc[np.arange(k, len(test), 22)].reset_index(drop = True) for k in range(22)]
    test_cat_players = np.stack([t.values for t in test_cat_players]).transpose(1,0, 2)

    test_cat_game = test_cat[cat_game_features].iloc[np.arange(0, len(test), 22)].reset_index(drop = True).values
#     test_cat_game = np.hstack([test_cat_game, test_cat[cat_player_features][test_dense["IsRusher"] > 0]])

    test_dense_players = np.reshape(test_dense_players, (len(test_dense_players), -1))
    test_dense = np.hstack([test_dense_players, test_dense_game])
    test_cat_players = np.reshape(test_cat_players, (len(test_cat_players), -1))
    test_cat = np.hstack([test_cat_players, test_cat_game])
    test_x = np.hstack([test_dense, test_cat])

    test_inp = test_x
    
    ## pred
    pred = 0
    for model in models:
        _pred = model.predict(test_inp)
        pred += _pred
    pred /= len(models)
    pred = np.clip(pred, 0, 1)
    env.predict(pd.DataFrame(data=pred,columns=sample.columns))
    return pred

In [ ]:
from kaggle.competitions import nflrush
env = nflrush.make_env()
preds = []
for test, sample in tqdm_notebook(env.iter_test()):
    pred = make_pred(test, sample, env, models)
    preds.append(pred)
env.write_submission_file()

In [ ]:
preds = np.vstack(preds)
## check whether prediction is submittable
print(np.mean(np.diff(preds, axis = 1) >= 0) == 1.0)
print(np.mean(preds > 1) == 0)

In [ ]:
print(losses)
print(np.mean(losses))